In [ ]:
!pip install fbprophet

In [ ]:
! pip install covid-data-api


#Import

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from tensorflow.python.keras.callbacks import EarlyStopping
from tensorflow.python.keras.callbacks import ModelCheckpoint
from tensorflow.python.keras.models import load_model
from statsmodels.tsa.arima_model import ARIMA
from fbprophet import Prophet
from fbprophet.plot import plot_plotly, add_changepoints_to_plot
import datetime
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import metrics

#Utils

In [ ]:
look_back=10
def creation_dataset_train_test(trainX, look_back,testX):
    trainX, trainY = create_dataset(trainX, look_back)
    testX, testY = create_dataset(testX, look_back)
    trainX = np.reshape(trainX, (trainX.shape[0], trainX.shape[1], 1))
    testX = np.reshape(testX, (testX.shape[0], testX.shape[1], 1))
    return trainX,testX, testY,testY

def scaling(y_deaths):
      # normalize the dataset
    scaler = MinMaxScaler(feature_range=(0, 1))
    y_deaths= scaler.fit_transform(y_deaths.reshape(-1,1))
    return y_deaths, scaler
def create_dataset(dataset, look_back):
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back)]
        dataX.append(a)
        dataY.append(dataset[i + look_back])
    return np.array(dataX), np.array(dataY)



def split_train_test(y_deaths):
    train_size = int(len(y_deaths) * 0.9)

    test_size = len(y_deaths) - train_size
    print("test_size", test_size)
    X=[]
    for i in range(len(y_deaths)):
       X.append(i)
    trainX,testX,trainY,testY=X[0:train_size],X[train_size:],y_deaths[0:train_size],y_deaths[-train_size:]
    print("len trzin , test ",len(trainX), len(testX))
    return trainX,testX,trainY,testY

import tensorflow as tf 
def train_eval_model_keras(norm,  df, nb_epochs, batch_size):
    y_deaths=df.values
    
    if (norm):
         y_deaths, scaler = scaling(df.values)
    trainX,testX,trainY,testY=split_train_test(y_deaths)
    trainX,testX, testY,testY = creation_dataset_train_test(trainX, look_back,testX)
    model = Sequential()
    model.add(LSTM(20, input_shape=(look_back, 1)))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    history=model.fit(trainX, trainY, epochs=nb_epochs, batch_size=batch_size , validation_split=0.2)
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')
    plt.ylabel('loss')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()
    score = model.evaluate(testX, testY, verbose=0)
    print( "Score d'evaluation", score )
    df=pd.DataFrame(columns =  ['yhat','y'])
    if (norm):
      
      df["y"]=scaler.inverse_transform(testY.reshape(-1,1)).reshape(-1)
      df["yhat"]=  scaler.inverse_transform(model.predict(testX).reshape(-1,1)).reshape(-1)
      #print("prediction data" ,df["yhat"].value_counts())
      #print( "df /n",df)
    else: 
      df["y"]=testY
      df["yhat"]=  model.predict(testX)
      #print(" prediction data" ,df["yhat"].value_counts())
      #print( "df /n",df)
    return df 


def fb_prophet_prediction(df):
    nb_future_day=60
    fb_df=pd.DataFrame(columns =  ['ds','y'])

    fb_df['ds']=df.index[:-nb_future_day]
    fb_df['y']=df.values[:-nb_future_day]
    
    test=fb_df.values[-nb_future_day:]
    from fbprophet import Prophet
    model = Prophet(interval_width =0.95)
    model.fit(fb_df)
    future  = model.make_future_dataframe(periods=nb_future_day)
    forecast_df= model.predict(future)
    forecast_df["true"]=df.values
    forecast_df=forecast_df[['ds','true','yhat','yhat_lower','yhat_upper']][-nb_future_day:]
    
    # Python
    fig1 = model.plot(forecast_df)
    return forecast_df


def get_prediction_using_ARIMA(df , colone):
        nb_days=60
        df_values = df.values[:-nb_days]
        data = pd.DataFrame(columns = ['ds','y'])
        data['ds']  = df.index[:-nb_days]
        data['y'] = df_values 
        arima = ARIMA(data['y'], order=(5, 1, 0))
        arima = arima.fit(trend='c', full_output=True, disp=True)
        forecast = arima.forecast(steps= nb_days)
        pred = list(forecast[0])
        prediction_dates= df.index[-nb_days:]
        print(len(pred), len(prediction_dates))
        print(df.index)
        
        df_arima= pd.DataFrame(columns = ['y_predicted','y_true'])
        df_arima['y_predicted']=pred
        df_arima['y_true']=df.values[-nb_days:]

        plt.figure(figsize= (15,10))
        plt.xlabel("Dates",fontsize = 20)
        plt.ylabel('Total cases',fontsize = 20)
        title= "ARIMA model : Predicted Values for " + str(colone)+ " for "+ str(nb_days) + " Days  "
        plt.title(title , fontsize = 20)

        plt.plot_date(y= pred,x= prediction_dates,linestyle ='dashed',color = '#ff9999',label = 'Predicted');
        print( prediction_dates)
        print(df.index)
        #plt.plot_date(y=data['y'],x=data['ds'],linestyle = '-',color = 'blue',label = 'Actual');
        plt.plot_date(y=df.values,x=df.index,linestyle = '-',color = 'green',label = 'Actual');
        plt.legend()
        return df_arima

# Extarction data from API

In [ ]:
from covid.api import CovId19Data

api = CovId19Data(force=False)

In [ ]:
import pandas as pd
from covid.api import CovId19Data
import re

class Extraction_data:
    def __init__(self,country_name, api):
        self.country_name=country_name
        self.api=api

    def data_extraction_from_api(self):

        data_from_api = self.api.get_history_by_country(self.country_name)

        # Getting dictionary
        data_dict = data_from_api  # json.dumps(data_from_api)
        return data_dict
    def data_preprocessing(self, data_dict):
        #data_preporcessing
        country_name = re.sub('[()\'\-,;]', '', self.country_name.lower())
        if (" " in country_name):
            country_name = "_".join(re.split(" ", country_name.lower()))

        # creating géo dictionary : optional
        """geo_data=dict()
        geo_data["country"]=data_dict[country_name]["label"] 
        geo_data["lat"]=data_dict[country_name]["lat"] 
        geo_data["long"]=data_dict[country_name]["long"] 
        print( "géo dictionary" ,geo_data)"""

        # getting historical data
        historical_data = data_dict[country_name]["history"]
        data_frame = pd.DataFrame.from_dict(historical_data, orient='index')
        return data_frame[["confirmed", "deaths", "recovered"]]

    def save_data_to_csv(self, data_frame):
        # save to csv
        filename = "data_" + self.country_name + ".csv"
        data_frame.to_csv(filename)


    def run(self):
        data_dict= self.data_extraction_from_api()
        data_frame=self.data_preprocessing( data_dict)
        self.save_data_to_csv(data_frame)
        return data_frame

# France Covid analysis

In [ ]:
france_data= Extraction_data("France", api).run()
print(france_data["confirmed"])

 ## Prediction for Confirmed cases

## LSTM

In [ ]:
df = train_eval_model_keras(norm=True, df=france_data["confirmed"], nb_epochs=30, batch_size=10)
df

In [ ]:
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df["y"],
                    mode='lines+markers',
                    name='confirmed',marker_color='purple'))
fig.add_trace(go.Scatter(x=df.index, y=df["yhat"],
                    mode='lines+markers',
                    name='confirmed_prediction', marker_color='green'))


fig.show()

In [ ]:
#calcule mse , 

mae = metrics.mean_absolute_error(df["y"], df["yhat"])
mse = metrics.mean_squared_error(df["y"], df["yhat"])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(df["y"], df["yhat"])

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

##Arima

In [ ]:
df_arima =get_prediction_using_ARIMA(france_data["confirmed"], "confirmed cases")

In [ ]:
mae = metrics.mean_absolute_error(df_arima["y_true"], df_arima["y_predicted"])
mse = metrics.mean_squared_error(df_arima["y_true"], df_arima["y_predicted"])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(df_arima["y_true"], df_arima["y_predicted"])

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

## Fbprophet

In [ ]:
  df_forcast = fb_prophet_prediction(france_data["confirmed"])
  


In [ ]:
mae = metrics.mean_absolute_error(df_forcast["true"], df_forcast["yhat"])
mse = metrics.mean_squared_error(df_forcast["true"], df_forcast["yhat"])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(df_forcast["true"], df_forcast["yhat"])

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

 ## Prediction for Deaths cases

##LSTM

In [ ]:
df = train_eval_model_keras(norm=True, df=france_data["deaths"], nb_epochs=30, batch_size=10)
df

In [ ]:
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df["y"],
                    mode='lines+markers',
                    name='confirmed',marker_color='purple'))
fig.add_trace(go.Scatter(x=df.index, y=df["yhat"],
                    mode='lines+markers',
                    name='confirmed_prediction', marker_color='green'))


fig.show()

In [ ]:
#calcule mse , 

mae = metrics.mean_absolute_error(df["y"], df["yhat"])
mse = metrics.mean_squared_error(df["y"], df["yhat"])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(df["y"], df["yhat"])

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

## Arima 

In [ ]:
df_arima =get_prediction_using_ARIMA(france_data["deaths"], "deaths cases")

In [ ]:
mae = metrics.mean_absolute_error(df_arima["y_true"], df_arima["y_predicted"])
mse = metrics.mean_squared_error(df_arima["y_true"], df_arima["y_predicted"])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(df_arima["y_true"], df_arima["y_predicted"])

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

## fbprophet

In [ ]:
df_forcast = fb_prophet_prediction(france_data["deaths"])
  

In [ ]:
mae = metrics.mean_absolute_error(df_forcast["true"], df_forcast["yhat"])
mse = metrics.mean_squared_error(df_forcast["true"], df_forcast["yhat"])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(df_forcast["true"], df_forcast["yhat"])

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

 ## Prediction for recovered cases

##LSTM

In [ ]:
df = train_eval_model_keras(norm=True, df=france_data["recovered"], nb_epochs=30, batch_size=10)

In [ ]:
import plotly.graph_objects as go
fig=go.Figure()
fig.add_trace(go.Scatter(x=df.index, y=df["y"],
                    mode='lines+markers',
                    name='confirmed',marker_color='purple'))
fig.add_trace(go.Scatter(x=df.index, y=df["yhat"],
                    mode='lines+markers',
                    name='confirmed_prediction', marker_color='green'))


fig.show()

## ARIMA

In [ ]:
df_arima =get_prediction_using_ARIMA(france_data["recovered"], "recovered cases")

In [ ]:
mae = metrics.mean_absolute_error(df_arima["y_true"], df_arima["y_predicted"])
mse = metrics.mean_squared_error(df_arima["y_true"], df_arima["y_predicted"])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(df_arima["y_true"], df_arima["y_predicted"])

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)

##Fbprophet

In [ ]:
df_forcast = fb_prophet_prediction(france_data["recovered"])
  

In [ ]:
mae = metrics.mean_absolute_error(df_forcast["true"], df_forcast["yhat"])
mse = metrics.mean_squared_error(df_forcast["true"], df_forcast["yhat"])
rmse = np.sqrt(mse) # or mse**(0.5)  
r2 = metrics.r2_score(df_forcast["true"], df_forcast["yhat"])

print("Results of sklearn.metrics:")
print("MAE:",mae)
print("MSE:", mse)
print("RMSE:", rmse)
print("R-Squared:", r2)